In [9]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, FloatSlider, IntSlider
import matplotlib.pyplot as plt

import numpy as np
import random

from gridsamp import gridsamp
from surrogate import Surrogate

random.seed(1234)
np.random.seed(1234)


## The effect of reguralization $r$ and spread $\lambda$ on $\hat{y}(\mathbf{x})$

In [8]:
from surrogate import scaling

# Constant data
def test_fun_1D(x):
    return -(1.4 - 3.0 * x) * np.sin(18.0 * x) + 2

# Plot bounds data
lb_spread = np.array([0.0])
ub_spread = np.array([1.2])
n_samples = 10

bounds_1d = np.array([lb_spread, ub_spread])
ns = [n_samples]
X_1d = np.linspace(bounds_1d[0, 0], bounds_1d[1, 0], ns[0])

# Training data
Y_1d = test_fun_1D(X_1d)

def plot_rbfs(r=0.0, λ=30.0):

    r = 10 ** (np.log10(1+1e-1)*r) - 1  # log scale  
    λ = 10 ** ((1+np.log10(500))*λ - 1)  # log scale 

    # train the model
    rbf1 = Surrogate(X=X_1d[:,None], Y=Y_1d[:,None], type="RBF", lb=lb_spread, ub=ub_spread, r=r, λ=λ, name="1D RBF good")
    rbf1.train()

    x_plot_1d = np.linspace(0, 1.2, 100)
    x_plot_1d_scaled = scaling(x_plot_1d,lb_spread,ub_spread,1)

    y_plot_1d = rbf1.predict(x_plot_1d.reshape(-1, 1))
    B = rbf1.model.basis(rbf1.model.λ,x_plot_1d_scaled)

    # Plot basis functions and test function
    fig, axs = plt.subplots(2, 1, figsize=(6.5, 6), sharex=True)

    # Assuming the previous plots
    axs[0].scatter(X_1d, Y_1d, label="samples")
    axs[0].plot(x_plot_1d, test_fun_1D(x_plot_1d), label="$y(x)$")
    axs[0].plot(x_plot_1d, y_plot_1d, label="$\hat{y}(x)$")
    axs[0].legend(loc="upper left")

    # Assuming B is already defined
    for i in range(B.shape[0]):
        axs[1].plot(x_plot_1d, B[i, :], label="_nolegend_")  # no legend for these lines
    axs[1].set_xlabel("x")
    axs[1].set_ylabel("$\phi (\lambda ||{\zeta -x_{i}}||)$")

    plt.tight_layout()
    plt.show()

# Create interactive plot with sliders for a and b
interact(plot_rbfs, r=FloatSlider(min=0, max=1, step=0.001, value=0.0), λ=FloatSlider(min=0, max=1, step=0.0001, value=0.5884));


interactive(children=(FloatSlider(value=0.0, description='r', max=1.0, step=0.001), FloatSlider(value=0.5884, …

## Effect of reguralization on ill-conditioned design matrices

In [17]:
import pandas as pd

scale = True

# read the data sets
data = pd.read_csv("sample_data_1.csv")
X_data_1 = data.iloc[:,2:4].to_numpy()
Y_data_1 = data.iloc[:,1:2].to_numpy()

data = pd.read_csv("sample_data_2.csv")
X_data_2 = data.iloc[:,2:4].to_numpy()
Y_data_2 = data.iloc[:,1:2].to_numpy()

# Plot bounds
lb_reg = np.array([1.0, 20.0])
ub_reg = np.array([12.0, 30.0])
bounds_reg = np.vstack([lb_reg, ub_reg])

n_grids = 200
n_reg = np.array([n_grids,n_grids])
X_predict_reg = gridsamp(bounds_reg,n_reg)

X_reg = X_predict_reg[:,0].reshape(n_reg)
Y_reg = X_predict_reg[:,1].reshape(n_reg)

# Define the function to plot
def plot_surfaces(d=1, r=0.0):
    r = 10 ** (np.log10(3)*r) - 1  # log scale

    ls1 = Surrogate(X=X_data_1, Y=Y_data_1, 
                    type="LS", lb=lb_reg, ub=ub_reg, r=r, d=d, scale=scale)  # use r from slider
    ls1.train()

    ls2 = Surrogate(X=X_data_2, Y=Y_data_2, 
                    type="LS", lb=lb_reg, ub=ub_reg, r=r, d=d, scale=scale)  # use r from slider
    ls2.train()

    # Predictions
    y_hat_1 = ls1.predict(X_predict_reg)
    y_hat_2 = ls2.predict(X_predict_reg)

    # Surface plot
    fig = plt.figure(figsize=(12, 5))

    ax1 = fig.add_subplot(121, projection='3d')
    ax1.plot_surface(Y_reg, X_reg, y_hat_1.reshape(Y_reg.shape), alpha=0.8)
    ax1.scatter(X_data_1[:, 1], X_data_1[:, 0], Y_data_1, c='r', label="samples")
    ax1.set_xlabel("x2")
    ax1.set_ylabel("x1")
    ax1.set_zlabel("$\hat{y}(x)$")

    ax2 = fig.add_subplot(122, projection='3d')
    ax2.plot_surface(Y_reg, X_reg, y_hat_2.reshape(Y_reg.shape), alpha=0.8)
    ax2.scatter(X_data_2[:, 1], X_data_2[:, 0], Y_data_2, c='r', label="samples")
    ax2.set_xlabel("x2")
    ax2.set_ylabel("x1")
    ax2.set_zlabel("$\hat{y}(x)$")

    plt.tight_layout()
    plt.show()

# Create interactive plot with sliders for d and r
interact(plot_surfaces, d=IntSlider(min=1, max=16, step=1, value=1), r=FloatSlider(min=0, max=1, step=0.001, value=0.0));


interactive(children=(IntSlider(value=1, description='d', max=16, min=1), FloatSlider(value=0.0, description='…